# H5 explorer

The h5explorer module provides a jupyter widget implementation for 
looking at 3 dimensional volumes in HDF5 files in a folder.

It allows the user to insert "filters" for modifying the volumes
and supports saving volumes in existing or new files.

In [ ]:
from feedWebGL2 import h5explorer

In [ ]:
E = h5explorer.Explorer()

# Adding custom filters

The explorer implementation allows the user to define custom
array transformation filters which define how to change a 3d volume array
using parameters which can be specified interactively.

The following example adds a simple (but not very useful) filter implementation
to an explorer object.  See the h5explorer.py source code for more examples
of filter implementations.

In [2]:
# Make a custom filter as a subclass of h5explorer.LogFilter
from feedWebGL2 import h5explorer

# Use standard Jupyter widget implementations to construct the interactive controls.
import ipywidgets as widgets

class FlipFilter(h5explorer.LogFilter):
    
    "Flip axes -- a simple filter example."

    # title is required
    title = "Flip"

    def widget(self):
        "The widget method defines the parameters for the filter."
        html1 = widgets.HTML("<H1>Flip one of the dimensions.</H1>")
        # self.info is required
        self.info = widgets.HTML("<div>Parameters</div>")
        # other parameters
        self.axis_name_dropdown = widgets.Dropdown(description="Axis to flip", options=["I", "J", "K"])
        # apply button is required
        self.button = widgets.Button(
            description="Flip it!",
        )
        self.button.on_click(self.apply)
        # Construct a widget container including all sub-widgets
        return widgets.VBox([html1, self.info, self.axis_name_dropdown, self.button])

    def apply(self, button):
        """
        The apply method defines how to execute the filter using the parameters.
        It must test that the image is defined.
        """
        # make sure an image has been loaded:
        ex = self.in_explorer
        image_array = ex.current_image
        if image_array is None:
            # no image: abort...
            self.update()
            return
        # Get parameters
        axis_name = self.axis_name_dropdown.value
        # apply the filter to the image
        if axis_name == "I":
            flipped = image_array[::-1, :, :]
        elif axis_name == "J":
            flipped = image_array[:, ::-1, :]
        elif axis_name == "K":
            flipped = image_array[:, :, ::-1]
        else:
            self.info.value = "??? Unknown axis name: " + repr(axis_name)
            return
        # store the modified array
        ex.current_image = flipped
        # redisplay all widgets
        ex.update_all()


In [3]:
# Configure an explorer with some the standard filters and also the FlipFilter

filter_classes = [
    h5explorer.LogFilter,
    h5explorer.BlurFilter,
    h5explorer.SubsampleFilter,
    FlipFilter,
]

CustomExplorer = h5explorer.Explorer(
    filters = filter_classes,
)